In [2]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# --------------------------------------------------------------------
# Daten laden
# --------------------------------------------------------------------

# Scatterplot-Daten (Kinos vs. Säle pro Kanton 2024)
df = pd.read_csv("Kinos_vs_Saele_2024.csv")   # enthält: Kanton, Kinos_2024, Saele_2024

# Liniendiagramm-Daten (Kinosäle nach Jahr, alle Kantone aufsummiert)
df_legende = pd.read_csv("Kinosäle_legende.csv", sep=";")

year_cols = [str(y) for y in range(2014, 2025)]
line_series = df_legende[year_cols].sum()

df_line = pd.DataFrame({
    "Year": line_series.index.astype(int),
    "Cinema Rooms": line_series.values
})

# Slider-Grenzen aus Sälen nehmen
slider_min = math.floor(df["Saele_2024"].min())
slider_max = math.ceil(df["Saele_2024"].max())


app.layout = html.Div(
    [
        html.H1("Dashboard 2"),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dcc.Dropdown(
                            options=["red", "green", "blue", "orange", "purple"],
                            value="red",
                            id="color",
                            multi=False,
                        )
                    ],
                    width=6,
                ),
                dbc.Col(
                    [
                        dcc.Slider(
                            min=slider_min,
                            max=slider_max,
                            id="min_value",
                        )
                    ],
                    width=6,
                ),
            ]
        ),
        dbc.Row(
            [
                dbc.Col([dcc.Graph(id="graph_1")], width=6),
                dbc.Col([dcc.Graph(id="graph_2")], width=6),
            ]
        ),
    ],
    className="m-4",
)

# --------------------------------------------------------------------
# Callbacks
# --------------------------------------------------------------------

# Liniendiagramm mit Dropdown-Farbe
@app.callback(Output("graph_1", "figure"), Input("color", "value"))
def update_graph_1(dropdown_value_color):
    fig = px.line(
        df_line,
        x="Year",
        y="Cinema Rooms",
        markers=True,
        color_discrete_sequence=[dropdown_value_color],
        labels={"Year": "Year", "Cinema Rooms": "Cinema Rooms"},
        title="Cinema Rooms 2014–2024",
    )
    fig.update_layout(template="plotly_white")
    return fig


# Scatterplot mit Slider-Filter
@app.callback(Output("graph_2", "figure"), Input("min_value", "value"))
def update_graph_2(min_value):
    # falls der Slider noch nicht bewegt wurde
    if min_value is None:
        min_value = slider_min

    dff = df[df["Saele_2024"] > min_value]

    fig = px.scatter(
        dff,
        x="Kinos_2024",
        y="Saele_2024",
        text="Kanton",
        labels={"Kinos_2024": "Kinos_2024", "Saele_2024": "Saele_2024"},
        title="Kinos vs. Kinosäle pro Kanton (2024)",
    )
    fig.update_traces(textposition="top center")
    fig.update_layout(template="plotly_white")
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8021)
